In [1]:
# import necessary libraries
from resemblyzer import preprocess_wav, VoiceEncoder
from  pathlib import Path 
from tqdm import tqdm 
import numpy as np 
# from Ipython.display import Audio 
from itertools import groupby
import heapq

# # run sample audio
# audio_sample = Audio("data/processed3/50_speakers_audio_data/Speaker_0000/Speaker_0000_Speaker_0000_00000_chunk0.wav")
# display(audio_sample)

In [2]:
import numpy as np
# import pandas as pd
import librosa
# import tensorflow as tf
# from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import os
import torch
import torchaudio
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [3]:

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # For CUDA
    torch.cuda.manual_seed_all(seed)  # For multi-GPU
    np.random.seed(seed)

    # Ensures deterministic behavior (optional, can slow things down)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [7]:
data_file= "data/raw/50_speakers_audio_data"

In [8]:
no_speakers_file=50

def speakers_list(no_speakers_file ,data_file ):
    speaker_l = []

    # Get all subfolders in the data_file directory
    subfolders = [f.name for f in os.scandir(data_file) if f.is_dir()]

    # Check if the requested number of speakers is available
    if no_speakers_file > len(subfolders):
        raise ValueError(f"Requested {no_speakers_file} speakers, but only {len(subfolders)} available.")

    # Select the first 'no_speakers_file' subfolders
    speaker_l = subfolders[:no_speakers_file]

    return speaker_l

speaker_list = speakers_list(no_speakers_file,data_file )

In [9]:
speaker_list

['Speaker_0006',
 'Speaker0028',
 'Speaker_0012',
 'Speaker0032',
 'Speaker0044',
 'Speaker_0008',
 'Speaker0036',
 'Speaker_0003',
 'Speaker0050',
 'Speaker0043',
 'Speaker_0018',
 'Speaker0046',
 'Speaker_0011',
 'Speaker0045',
 'Speaker_0014',
 'Speaker0029',
 'Speaker0040',
 'Speaker_0005',
 'Speaker_0022',
 'Speaker_0025',
 'Speaker0035',
 'Speaker_0017',
 'Speaker0026',
 'Speaker_0013',
 'Speaker_0010',
 'Speaker0049',
 'Speaker_0024',
 'Speaker0048',
 'Speaker0031',
 'Speaker_0015',
 'Speaker_0020',
 'Speaker0030',
 'Speaker_0000',
 'Speaker0037',
 'Speaker0047',
 'Speaker_0023',
 'Speaker_0016',
 'Speaker0041',
 'Speaker0038',
 'Speaker_0001',
 'Speaker_0007',
 'Speaker_0019',
 'Speaker0034',
 'Speaker0027',
 'Speaker0033',
 'Speaker0042',
 'Speaker_0009',
 'Speaker0039',
 'Speaker_0021',
 'Speaker_0002']

In [10]:
import os
import numpy as np
from resemblyzer import preprocess_wav, VoiceEncoder
from tqdm import tqdm
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance

# Set up Qdrant client
client = QdrantClient(url="http://localhost:6333")
embeddings = []
collection_name="my-collection"
encoder = VoiceEncoder("cuda")

# Define function to preprocess and encode audio files
def process_wavs(data_folder, speaker_list, collection_name="my-collection"):
    # Initialize encoder
   

    # Prepare storage for embeddings
   

    # Iterate through each speaker and their files
    for speaker_id, speaker_folder in enumerate(speaker_list):
        folder_path = os.path.join(data_folder, speaker_folder)
        
        # Process each audio file
        for file_name in tqdm(os.listdir(folder_path), desc=f"Processing {speaker_folder}"):
            if file_name.endswith('.wav'):
                file_path = os.path.join(folder_path, file_name)
                
                # Preprocess and encode the wav
                wav = preprocess_wav(file_path)
                embedding = encoder.embed_utterance(wav)
                
                # Add embedding to the list with speaker info
                embedding_dict = {
                    "id": len(embeddings) + 1,  # Unique ID for each embedding
                    "vector": embedding.tolist(),
                    "payload": {"speaker_id": int(speaker_folder[-2:]), "speaker_folder": speaker_folder}
                }
                embeddings.append(embedding_dict)



Loaded the voice encoder model on cuda in 0.18 seconds.


In [11]:
embeddings

[]

In [12]:

# Example usage
process_wavs(data_file, speaker_list)

Processing Speaker_0003:  11%|█         | 9/85 [00:01<00:12,  6.11it/s]/data/anaconda3/envs/speech/lib/python3.13/site-packages/resemblyzer/audio.py:104: RuntimeWarning: divide by zero encountered in log10
  wave_dBFS = 20 * np.log10(rms / int16_max)
/data/anaconda3/envs/speech/lib/python3.13/site-packages/resemblyzer/audio.py:108: RuntimeWarning: invalid value encountered in multiply
  return wav * (10 ** (dBFS_change / 20))
/data/anaconda3/envs/speech/lib/python3.13/site-packages/resemblyzer/audio.py:72: RuntimeWarning: invalid value encountered in cast
  pcm_wave = struct.pack("%dh" % len(wav), *(np.round(wav * int16_max)).astype(np.int16))
Processing Speaker0043:  21%|██        | 7/34 [00:01<00:05,  5.10it/s][src/libmpg123/layer3.c:INT123_do_layer3():1774] error: part2_3_length (1696) too large for available bit count (1568)
[src/libmpg123/layer3.c:INT123_do_layer3():1774] error: part2_3_length (1600) too large for available bit count (1568)
Processing Speaker_0002: 100%|██████████

In [13]:
collection_name="speaker_recognition_testing_raw"

In [16]:

    # Define the collection in Qdrant if it doesn't exist
client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=256, distance=Distance.COSINE),
)

UnexpectedResponse: Unexpected Response: 409 (Conflict)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `speaker_recognition_testing_raw` already exists!"},"time":0.011504728}'

In [17]:
len(embeddings[0]["payload"]),embeddings[2008]["id"]

(2, 2009)

In [18]:
train_embed,test_embed=train_test_split(embeddings, test_size=0.2, random_state=42)

In [19]:
from qdrant_client.models import PointStruct
train_points = [
    PointStruct(
        id=embedding["id"], 
        vector=embedding["vector"], 
        payload=embedding.get("payload")
    ) 
    for embedding in train_embed
]
test_points = [
    PointStruct(
        id=embedding["id"], 
        vector=embedding["vector"], 
        payload=embedding.get("payload")
    ) 
    for embedding in test_embed
]

In [20]:
batch_size = 1000
for i in range(0, len(train_points), batch_size):
    batch = train_points[i:i + batch_size]
    client.upsert(collection_name=collection_name, points=batch)

In [15]:
test_wav = preprocess_wav("src/Backend/audio.wav")
test_wav2 = preprocess_wav("data/processed3/50_speakers_audio_data/Speaker_0000/Speaker_0000_Speaker_0000_00000_chunk0.wav")
# Create a voice encoder object
encoder = VoiceEncoder("cuda")
test_embeddings = encoder.embed_utterance(test_wav)
test_embeddings2 = encoder.embed_utterance(test_wav2)
# Search related embeddings
results = client.search("speaker_recognition", test_embeddings.tolist())
print(results)

/data/anaconda3/envs/speech/lib/python3.13/site-packages/resemblyzer/audio.py:27: UserWarning: PySoundFile failed. Trying audioread instead.
  wav, source_sr = librosa.load(str(fpath_or_wav), sr=None)
/data/anaconda3/envs/speech/lib/python3.13/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Loaded the voice encoder model on cuda in 0.01 seconds.


/tmp/ipykernel_2170985/2055009604.py:8: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search("speaker_recognition", test_embeddings.tolist())


[ScoredPoint(id=36724, version=77, score=0.79123366, payload={'speaker_id': 7, 'speaker_folder': 'Speaker_0007'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=19130, version=87, score=0.75000334, payload={'speaker_id': 17, 'speaker_folder': 'Speaker_0017'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=47380, version=56, score=0.71510196, payload={'speaker_id': 4, 'speaker_folder': 'Speaker_0004'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=36960, version=36, score=0.71390367, payload={'speaker_id': 7, 'speaker_folder': 'Speaker_0007'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=17423, version=72, score=0.7130449, payload={'speaker_id': 5, 'speaker_folder': 'Speaker_0005'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=16088, version=85, score=0.70682555, payload={'speaker_id': 5, 'speaker_folder': 'Speaker_0005'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=36803, version=64,

In [13]:
np.dot(np.array(test_wav), np.array(test_wav2))
# Calculate cosine similarity
cosine_similarity = np.dot(test_embeddings, test_embeddings2) / (np.linalg.norm(test_embeddings) * np.linalg.norm(test_embeddings2))
print("Cosine Similarity:", cosine_similarity)

Cosine Similarity: 0.72377324


In [42]:
correct = 0
total = len(test_embed)
incorrect = []
for embed in test_embed:
     # Search related embeddings
    results = client.search("speaker_recognition_testing", embed["vector"])
    if(len(results) > 0):
        if results[0].payload["speaker_id"] == embed["payload"]["speaker_id"]:
            correct += 1
        else :
            incorrect.append(embed)

/tmp/ipykernel_1812508/1427623008.py:6: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search("speaker_recognition_testing", embed["vector"])


In [43]:
correct, total, correct/total

(9324, 9755, 0.9558175294720656)

In [44]:
print(f"Accuracy: {correct/total:.2%}")

Accuracy: 95.58%


In [21]:
from collections import Counter


correct = 0
total = len(test_embed)
incorrect = []
for embed in test_embed:
     # Search related embeddings
    results = client.search(collection_name, embed["vector"], score_threshold=0.6)
    
    # Extract speaker IDs from the top 5 results
    top_5_speaker_ids = [result.payload["speaker_id"] for result in results]

    # Count the occurrences of each speaker ID
    speaker_id_counts = Counter(top_5_speaker_ids)

    # Find the speaker ID with the maximum count
    most_frequent_speaker_id = speaker_id_counts.most_common(1)[0][0]

    if(len(results) > 0):
        if most_frequent_speaker_id == embed["payload"]["speaker_id"]:
            correct += 1
        # else :
        #     incorrect.append(embed)
        
correct, total, correct/total

/tmp/ipykernel_2170985/2830998579.py:9: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(collection_name, embed["vector"], score_threshold=0.6)


(479, 485, 0.9876288659793815)

In [22]:
print(f"Accuracy: {correct/total:.2%}")

Accuracy: 98.76%


In [64]:
from collections import defaultdict


correct = 0
total = len(test_embed)
incorrect = []
for embed in test_embed:
     # Search related embeddings
    results = client.search("speaker_recognition_testing", embed["vector"], score_threshold=0.6)
    
    # Group scores by speaker_id
    score_sums = defaultdict(float)
    for result in results:
        speaker_id = result.payload["speaker_id"]
        score_sums[speaker_id] += result.score

    # Find the speaker_id with the maximum cumulative score
    predicted_speaker_id = max(score_sums, key=score_sums.get)

    if(len(results) > 0):
        if predicted_speaker_id == embed["payload"]["speaker_id"]:
            correct += 1
        # else :
        #     incorrect.append(embed)
        
print(f"Accuracy: {correct/total:.2%}")

/tmp/ipykernel_1812508/861127245.py:9: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search("speaker_recognition_testing", embed["vector"], score_threshold=0.6)


Accuracy: 96.38%


In [46]:
for embed in incorrect:
     # Search related embeddings
    results = client.search("speaker_recognition_testing", embed["vector"])
    print(f"retrived : {results[0].payload["speaker_id"]}, true: {embed["payload"]["speaker_id"]}")

/tmp/ipykernel_1812508/911448685.py:3: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search("speaker_recognition_testing", embed["vector"])


retrived : 4, true: 0
retrived : 4, true: 0
retrived : 6, true: 2
retrived : 6, true: 2
retrived : 5, true: 4
retrived : 9, true: 2
retrived : 4, true: 0
retrived : 2, true: 6
retrived : 36, true: 45
retrived : 4, true: 0
retrived : 6, true: 2
retrived : 4, true: 0
retrived : 6, true: 2
retrived : 2, true: 6
retrived : 6, true: 2
retrived : 2, true: 6
retrived : 0, true: 4
retrived : 6, true: 2
retrived : 6, true: 2
retrived : 6, true: 2
retrived : 2, true: 6
retrived : 0, true: 4
retrived : 38, true: 20
retrived : 20, true: 38
retrived : 20, true: 38
retrived : 33, true: 39
retrived : 8, true: 6
retrived : 6, true: 2
retrived : 45, true: 36
retrived : 3, true: 39
retrived : 6, true: 2
retrived : 25, true: 14
retrived : 2, true: 6
retrived : 2, true: 6
retrived : 2, true: 6
retrived : 6, true: 2
retrived : 2, true: 6
retrived : 2, true: 6
retrived : 20, true: 38
retrived : 6, true: 2
retrived : 2, true: 6
retrived : 3, true: 4
retrived : 2, true: 6
retrived : 44, true: 16
retrived : 14

In [37]:
results = client.search("speaker_recognition_testing", np.array(embed["vector"]))

/tmp/ipykernel_1812508/355063134.py:1: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search("speaker_recognition_testing", np.array(embed["vector"]))


In [38]:
results

[ScoredPoint(id=25713, version=37, score=0.89749545, payload={'speaker_id': 31, 'speaker_folder': 'Speaker0031'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=25311, version=9, score=0.8848518, payload={'speaker_id': 31, 'speaker_folder': 'Speaker0031'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=25898, version=13, score=0.8843004, payload={'speaker_id': 31, 'speaker_folder': 'Speaker0031'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=25824, version=23, score=0.88186425, payload={'speaker_id': 31, 'speaker_folder': 'Speaker0031'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=25567, version=33, score=0.8786396, payload={'speaker_id': 31, 'speaker_folder': 'Speaker0031'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=25380, version=16, score=0.8781756, payload={'speaker_id': 31, 'speaker_folder': 'Speaker0031'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=25732, version=38

In [1]:
from resemblyzer import preprocess_wav, VoiceEncoder
from qdrant_client import QdrantClient
from collections import Counter


def pred_speaker(wav_path: str, collection_name: str = "speaker_recognition_testing",url: str = "http://localhost:6333"): 
    client = QdrantClient(url=url)
    encoder = VoiceEncoder("cuda")
    test_wav = preprocess_wav(wav_path)
    test_embeddings = encoder.embed_utterance(test_wav)
    results = client.search(collection_name, test_embeddings, score_threshold=0.6)
    top_5_speaker_ids = [result.payload["speaker_id"] for result in results]
    speaker_id_counts = Counter(top_5_speaker_ids)
    most_frequent_speaker_id = speaker_id_counts.most_common(1)[0][0]
    return most_frequent_speaker_id


In [2]:
predicted_speaker_id = pred_speaker("/home/raid3/Diwanshu/prml/prml_speaker_recog_spring_2025/data/processed3/50_speakers_audio_data/Speaker_0000/Speaker_0000_Speaker_0000_00000_chunk1.wav")

Loaded the voice encoder model on cuda in 6.08 seconds.


/tmp/ipykernel_441694/3167145769.py:11: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(collection_name, test_embeddings, score_threshold=0.6)


ResponseHandlingException: [Errno 111] Connection refused

In [28]:
predicted_speaker_id

0